# P6 → Ocean thermal and haline change contributions to Sea Level trends

## Introduction
Sea level increases because of changes in currents (dynamic effect) and because of ocean density changes (steric effect). Hence, Sea surface height increases not only as water is added or transported, but also as water warms and its volume expands. Changes in salt content, or salinity, also affect sea levels. These are known as “thermosteric” and “halosteric” changes. This project is about caracterising and looking at the steric contribution to Sea Level Change.

You will need to compute ocean density changes contribution to Sea level rises (thermosteric and halosteric effects) and demonstrate that it is the driver of regional sea level change trends.

*Bibliography*:

- [Ocean and climate scientific sheet](https://ocean-climate.org/wp-content/uploads/2015/03/sea-level_ScientificItems_BD-3.pdf)
- [Overview](https://sealevel.nasa.gov/understanding-sea-level/overview)
- [Deep-ocean contribution to sea level and energy budget not detectable over the past decade](https://www.nature.com/articles/nclimate2387)
- [Last IPCC report on Sea Level changes](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_Chapter_09.pdf#page=55)
- [IPCC fig 9.12](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_Chapter_09.pdf#page=237)
- [Meyssignac et al, 2012](https://www.sciencedirect.com/science/article/pii/S0264370712000464)

*Ideas for your project*

Below is a list of what you could do in a timely manner in your project.

You shall first compute the Sea Level trend and remove its global average. This local trend will be the pattern to explain.

Then you should compute the thermal and haline contribution to steric height. To do so you will use the GSW librairy that will help you compute the absolute salinity, conservative temperature and them the thermal expansion and haline contraction coefficient.

You may compare the Sea Level trend from altimetry with the steric height from observations. Be careful to compare similar time period.

Using the bibliography, you may consider to compare your results with already published ones, to ensure the accuracy of your computation.

The EN4 dataset to work with is a global interpolation of all available ocean in-situ observations. You could further compare CMIP6 projections or historical simulations to the EN4 reference.

## Description

The Steric contribution to Sea Level Anomaly (SSLA) is computed as the vertical integral of density anomalies:
$$SSLA = \frac{-1}{\rho_0}\int_{-H}^{0} \Delta \rho \,dz$$
which can be further decomposed into thermosteric (temperature) and halosteric (salinity) contributions:
$$SSLA = TSLA + HSLA$$
with:
$$TSLA = -\int_{-H}^{0} \alpha \Delta T \,dz$$
$$HSLA =  \int_{-H}^{0} \beta \Delta S \,dz$$

where:
- $\Delta \rho$ the density anomaly, refered to a climatic mean,
- $\Delta T$ the temperature anomaly, refered to a climatic mean,
- $\Delta S$ the salinity anomaly, refered to a climatic mean,
- $\rho_0$ is the reference density $1025 kg/m^{3}$, 
- $\alpha$ is the ocean thermal expansion coefficient,
- $\beta$ the haline contraction coefficient,
- $H$ is the reference depth, set to 700, 1000 or 2000m.

$\alpha$ and $\beta$ can be computed with the [GSW library](https://teos-10.github.io/GSW-Python/).

Note that you should take of the irregular area and thickness of the grid cells when computing horizontal or vertical averages/integrals ([see the P5 notebook](https://github.com/obidam/ds2-2025/blob/main/projects/P5-OceanWarming-for-students.ipynb)).

All ocean variables can be obtained directly from the EN4 dataset. Be careful with units. The Sea Level Change from altimetry data is obtainted from the AVISO dataset, also provided by the class data catalogue.

The main problem in this project is to calculate $TSLA$ and $HSLA$.

# Getting started

## Load all the necessary libraries

In [ ]:
import os, sys, urllib, tempfile
with tempfile.TemporaryDirectory() as tmpdirname:
    sys.path.append(tmpdirname)
    repo = "https://raw.githubusercontent.com/obidam/ds2-2025/main/"
    urllib.request.urlretrieve(os.path.join(repo, "utils.py"), 
                               os.path.join(tmpdirname, "utils.py"))
    from utils import check_up_env
    ds2tools = check_up_env(with_tuto=True)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
import dask

from intake import open_catalog
import gsw

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
create_map = ds2tools.create_map

## Connect to the data catalog

In [ ]:
catalog_url = 'https://raw.githubusercontent.com/obidam/ds2-2025/main/ds2_data_catalog.yml'
cat = open_catalog(catalog_url)
cat

In [ ]:
# Load ocean dataset EN4
en4 = cat["en4"].to_dask()
print("Size of the dataset:", en4.nbytes/1e9,"Gb")
en4

In [ ]:
# Load Sea Level Anomalies from altimetry:
ssh = cat["sea_surface_height"].to_dask()
print("Size of the dataset:", ssh.nbytes/1e9,"Gb")
ssh

## Connect to a Coiled Dask cluster

For computational expensive operation, you better connect to the Coiled Dask cluster available to the class:

In [ ]:
import coiled
cluster = coiled.Cluster(name="ds2-highcpu-binder", workspace="class-2025")
# cluster = coiled.Cluster(name="ds2-highmem-binder", workspace="class-2025")
client = cluster.get_client()

# Sea Level Change from Altimetry
Before moving on to computing TSLA and HSLA, you can compute time series and maps of the SSH variable to familiarize with it.

### Surface elements for global horizontal averaging
To compute a global mean, you need to weight local values with the grid cell areas, in order to account for the latitude dependance of the grid.

We'll be using the formula for the area element in spherical coordinates. The [area element for lat-lon coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system#Integration_and_differentiation_in_spherical_coordinates) is

$$ \delta A (x,y) = R^2 \delta \phi \delta \lambda \cos(\phi) $$

where $\phi$ is latitude, $\delta \phi$ is the spacing of the points in latitude, $\delta \lambda$ is the spacing of the points in longitude, and $R$ is Earth's radius. (In this formula, $\phi$ and $\lambda$ are measured in radians.) Let's use xarray to create the weight factor.

Create a function that compute the surface element $\delta A$.

In [ ]:
def get_dA(ds):
    # Earth Radius in meters:
    R = 6.37e6  
    # we know already that the spacing of the points is one degree latitude:
    dϕ = np.deg2rad(1.)
    dλ = np.deg2rad(1.)
    # Apply formulae for area element for lat-lon coordinates:
    dA = 
    return xr.DataArray(dA, dims='latitude', name='dA', attrs={'unit': 'm2', 'long_name': 'Grid surface elements'})

ssh['dA'] = get_dA(ssh)
ssh['dA'].plot()

We can now compute the sea level anomaly time series:

In [ ]:
%%time
gsla = ssh['sla'].weighted(ssh['dA']).mean(dim=['latitude', 'longitude'])
gsla

In [ ]:
gsla.plot()

### Local Sea Level Trends

What we're realy interested in the local trend of SLA.

To compute local trends, we will let xarray handle the latitude and longitude dimensions and simply works with xarray DataArrays.

Local trends can be computed as the ratio of the time/sla covariance with time variance. 

So let's define the following sum of squares:

for time:
$$ss_{tt} = \Sigma_{i=1}^{N} (t_i - \bar{t})^2$$

for sla:
$$ss_{hh} = \Sigma_{i=1}^{N} (h_i -\bar{h})^2$$

for time/sla:
$$ss_{th} = \Sigma_{i=1}^{N} (t_i - \bar{t})(h_i -\bar{h})$$

Then, the slope of the linear least square fit is simply given by:
$$a = \frac{ss_{th}}{ss_{tt}}$$

If we further estimate the variance for the fit error as:
$$s^2 = \frac{ss_{hh} - a ss_{th}}{N-2}$$

the standard error on the slope $a$ is given by:
$$SE(a) = \frac{s}{\sqrt{ss_{tt}}}$$

Let's now compute all the required sum of squares.

Note that for time, we need to convert the numpy datetime format to a more numerical value, like julian days.

In [ ]:
ds = ssh.where(ssh['time']>=pd.to_datetime('19930101'), drop=True).where(ssh['time']<=pd.to_datetime('20100101'), drop=True)
ds

In [ ]:
%%time
# For time:
ds['t'] = xr.DataArray(ds['time'].to_pandas().index.to_julian_date().values, dims='time', name='juld')
sstt = ((ds['t']-ds['t'].mean(dim='time'))**2).sum(dim='time')

# For sla:
sshh = ((ds['sla']-ds['sla'].mean(dim='time'))**2).sum(dim='time').compute().persist()

# For time vs sla:
ssth = (((ds['t'] - ds['t'].mean(dim='time')) * (ds['sla'] - ds['sla'].mean(dim='time'))).sum(dim='time')).compute().persist()

Let's compute the slope of the linear least square fit:

In [ ]:
%%time
ssh['sla_trend'] = ssth / sstt  # m / days
ssh['sla_trend'].compute().persist()

And finaly remove the global sea level trend to get only local anomalies

In [ ]:
global_sla_trend = ssh['sla_trend'].weighted(ssh['dA']).mean()
global_sla_trend*365*1000  # mm/year

Let's now make a nice map of the local SLA trend:

In [ ]:
fig, proj, ax = create_map()
# fig, proj, ax = create_map(extent=[-90, 0, 0, 80])

((ssh['sla_trend']-global_sla_trend)*365*1000).plot(transform=proj, ax=ax, 
                     levels=15, vmin=-14, vmax=14, 
                     cmap=mpl.colormaps.get_cmap('bwr'),                     
                     cbar_kwargs={'shrink': 0.8})
ax.add_feature(cfeature.LAND, facecolor=[0.7]*3, zorder=100)
ax.set_title("Local Sea Level Anomaly trend in mm/year")

# Steric contributions to SLA trends

We shall now focus on the core of the P6 project: 
$$TSLA = -\int_{-H}^{0} \alpha \Delta T \,dz$$
$$HSLA =  \int_{-H}^{0} \beta \Delta S \,dz$$

## Grid elements

But we first need the grid elements to compute the vertical and horizontal averages correctly:

In [ ]:
def get_dA(ds):
    # Earth Radius in meters:
    R = 6.37e6  
    # we know already that the spacing of the points is one degree latitude:
    dϕ = np.deg2rad(1.)
    dλ = np.deg2rad(1.)
    # Apply formulae for area element for lat-lon coordinates:
    dA = 
    return xr.DataArray(dA, dims='lat', name='dA', attrs={'unit': 'm2', 'long_name': 'Grid surface elements'})

def get_dz(ds):
    depth = ds['depth'].values
    dz_0 = depth[0] + (depth[1]-depth[0])/2
    dz_i = (depth[1:-1]-depth[:-2])/2 + (depth[2:]-depth[1:-1])/2
    dz_N = (depth[-1]-depth[-2])/2
    dz = np.concatenate((dz_0[np.newaxis], dz_i, dz_N[np.newaxis]))
    return xr.DataArray(dz, dims='depth', name='dZ', attrs={'unit': 'm', 'long_name': 'Grid vertical thickness'})

en4['dA'] = get_dA(en4);
en4['dZ'] = get_dz(en4);

## Thermal and haline coefficients

Now let's use the GSW library to compute the absolute salinity and conservative temperature:

In [ ]:
en4['pres'] = gsw.p_from_z(-en4['depth'], en4['lat'])
en4['pres'].compute().persist()

en4['SA'] = gsw.SA_from_SP(en4['salinity'], en4['pres'], en4['lon'], en4['lat'])
en4['SA'].attrs = {'unit': 'g/kg', 'long_name': 'Absolute Salinity'}

en4['CT'] = gsw.CT_from_t(en4['SA'], (en4['temperature']-273.15), en4['pres'])
en4['CT'].attrs = {'unit': 'degC', 'long_name': 'Conservative Temperature of seawater from in-situ temperature'}

and finaly we can compute the $\alpha$ and $\beta$ coefficients:

In [ ]:
en4['alpha'] = 
en4['alpha'].attrs = {'unit': '1/K', 'long_name': 'Thermal expansion coefficient'}

en4['beta'] = 
en4['beta'].attrs = {'unit': 'kg/g', 'long_name': 'Haline contraction coefficient'}

Let's have a look at those coefficients

Check should compare to:

https://www.science.org/doi/10.1126/sciadv.abq0793#:~:text=In%20the%20global%20ocean%2C%20the,C%E2%88%921%20in%20tropical%20waters.

https://en.wikipedia.org/wiki/Haline_contraction_coefficient#/media/File:Beta_30W.png

In [ ]:
%%time
da = en4['alpha'].isel(depth=0).weighted(en4['dA']).mean(dim=['time', 'lon'])
da.attrs = en4['alpha'].attrs

In [ ]:
(da/1e-4).plot(y='lat')

In [ ]:
%%time
da = en4['beta'].sel(lon=360-30, method='nearest').weighted(en4['dA']).mean(dim=['time']).compute().persist()
da.attrs = en4['beta'].attrs

In [ ]:
da.plot(yincrease=False, levels=12, figsize=(10,5))

## Steric heights

In order to compute the steric height contribution from changes in temperature or salinity, we need a reference level for the integration:

In [ ]:
H =   # in meters

In [ ]:
%%time
en4['DT'] = en4['temperature'] - en4['temperature'].mean(dim='time')
en4['TSLA'] = 
en4['TSLA'] = en4['TSLA'].compute().persist()
en4['TSLA'].attrs = {'unit': 'm', 'long_name': 'Thermosteric height'}

In [ ]:
%%time
en4['DS'] = en4['SA'] - en4['SA'].mean(dim='time')
en4['HSLA'] = 
en4['HSLA'] = en4['HSLA'].compute().persist()
en4['HSLA'].attrs = {'unit': 'm', 'long_name': 'Halosteric height'}

## Local trends of steric height components

In [ ]:
def compute_trend(ds, vname, time='time'):
    # For time:
    ds['t'] = xr.DataArray(ds[time].to_pandas().index.to_julian_date().values, dims=time, name='juld')
    sstt = ((ds['t']-ds['t'].mean(dim=time))**2).sum(dim=time)

    # For vname:
    ssyy = ((ds[vname]-ds[vname].mean(dim=time))**2).sum(dim=time)

    # For time vs vname:
    ssty = (((ds['t'] - ds['t'].mean(dim=time)) * (ds[vname] - ds[vname].mean(dim=time))).sum(dim=time))

    # Trend:
    trend = ssty / sstt  # [unit] / days
    
    return trend

Let's define the time period to look at:

1993-2010 for instance, to compare with results from [Meyssignac et al, 2012](https://www.sciencedirect.com/science/article/pii/S0264370712000464)


In [ ]:
# ds = en4.where(en4['time']>=ssh['time'].min(), drop=True).where(en4['time']<=ssh['time'].max(), drop=True)
ds = en4.where(en4['time']>=pd.to_datetime('19930101'), drop=True).where(en4['time']<pd.to_datetime('20110101'), drop=True)

and compute trends:

In [ ]:
%%time
ds['HSLA_trend'] = compute_trend(ds, 'HSLA').compute().persist()
ds['TSLA_trend'] = compute_trend(ds, 'TSLA').compute().persist()

In [ ]:
global_steric_trend = (ds['TSLA_trend']+ds['HSLA_trend']).weighted(ds['dA'].fillna(0)).mean()
global_steric_trend*365*1000  # mm/year

In [ ]:
fig, proj, ax = create_map()
((ds['TSLA_trend']+ds['HSLA_trend']-global_steric_trend)*365*1000).plot(transform=proj, ax=ax,   
                     levels=15, vmin=-14, vmax=14, 
                     cmap=mpl.colormaps.get_cmap('bwr'))
ax.add_feature(cfeature.LAND, facecolor=[0.7]*3, zorder=100)
ax.set_title("Local Steric Sea Level Anomaly trend in mm/year")

## Compare steric trend to altimetry

You shall now interpolate all sea level trends and compare them on maps



In [ ]:
fig, proj, ax = create_map()
# fig, proj, ax = create_map(extent=[-90, 0, 0, 80])

((ssh['sla_trend']-global_sla_trend)*365*1000).plot(transform=proj, ax=ax, 
                     levels=15, vmin=-14, vmax=14, 
                     cmap=mpl.colormaps.get_cmap('bwr'),                     
                     cbar_kwargs={'shrink': 0.8})
ax.add_feature(cfeature.LAND, facecolor=[0.7]*3, zorder=100)
ax.set_title("Local Sea Level Anomaly trend in mm/year")

## Timeseries